In [1]:
import sys
sys.path.append("./")
sys.path.append("./hw2/stud/")
from torch.utils.data import DataLoader
import config
# from tqdm.auto import tqdm
import numpy as np
from utils import *
from load import *
# from pytorch_lightning.callbacks import ModelCheckpoint
# from pytorch_lightning import Trainer
from glossBERT import GlossBERT


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# FINE_TRAIN_DATA
# FINE_VAL_DATA
# FINE_TEST_DATA


In [3]:
print(config.DEVICE)

cuda


In [4]:
seed_everything(config.SEED)

In [5]:

train_dataset = FineGrainedDataset(config.FINE_TRAIN_DATA)
val_dataset = FineGrainedDataset(config.FINE_VAL_DATA)
test_dataset = FineGrainedDataset(config.FINE_TEST_DATA)

train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True, collate_fn=glossBERT_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE, shuffle=False, collate_fn=glossBERT_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE, shuffle=False, collate_fn=glossBERT_collate_fn)

mapping = load_map(config.MAP_PATH)


In [6]:
# take a look at the data 
#load one data fr om the train_dataset
print(train_dataset[0])

(['<s>', 'Bailly', ',', 'after', 'leaving', 'Fort Snelling', 'in', 'August', '1821', ',', 'was', 'forced', 'to', 'leave', 'some', 'of', 'the', 'cattle', 'at', 'the', "Hudson's Bay Company's", 'post', 'on', 'Lake Traverse', '``', 'in', 'the', 'Sieux Country', "''", 'and', 'reached', 'Fort Garry', ',', 'as', 'the', 'Selkirk', "Hudson's Bay Company", 'center', 'was', 'now', 'called', ',', 'late', 'in', 'the', 'fall', '.', 'He', 'set out', 'on', 'his', '700', '-', 'mile', 'return', 'journey', 'with', 'five', 'families', 'of', 'discontented', 'and', 'disappointed', 'Swiss', 'who', 'turned', 'their', 'eyes', 'toward', 'the', 'United States', '.', '</s>', 'mile:', 'a', 'unit', 'of', 'length', 'used', 'in', 'navigation;', 'exactly', '1,852', 'meters;', 'historically', 'based', 'on', 'the', 'distance', 'spanned', 'by', 'one', 'minute', 'of', 'arc', 'in', 'latitude', '</s>'], ['person', ',', 'after', 'leave', 'location', 'in', 'august', '1821', ',', 'be', 'force', 'to', 'leave', 'some', 'of', 't

In [7]:
# # # display one item
# for batch in train_loader:
#     input_ids, senses_ids, labels, attention_mask = batch
#     # pritn one item
#     print(input_ids)
#     print(senses_ids)
#     print(labels)
#     print(attention_mask)

#     break



In [9]:
model = GlossBERT()
# model.to(config.DEVICE)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able

In [11]:
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger

# TRAINER
# wandb_logger = WandbLogger(project='nlp_hw2', log_model=True)
trainer = Trainer(gpus=0, max_epochs=10, callbacks=[ModelCheckpoint(monitor='val_loss')])#, logger=wandb_logger)
trainer.fit(model, train_loader, val_loader)
# trainer = Trainer(max_epochs=10, callbacks=[ModelCheckpoint(monitor='val_loss')])
# trainer.fit(model, train_loader, val_loader)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/francesco/miniconda3/envs/nlp2023-hw2/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1566: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  rank_zero_warn(

  | Name       | Type                          | Params
-------------------------------------------------------------
0 | roberta    | RobertaForTokenClassification | 124 M 
1 | bert       | RobertaModel                  | 124 M 
2 | relu       | ReLU                          | 0     
3 | classifier | Linear                        | 1.5 K 
4 | loss       | CrossEntropyLoss              | 0     
-------------------------------------------------------------
248 M     Trainable params
0         Non-trainable params
248 M     Total params
994.815   Total estimated model params size (MB)


Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/home/francesco/miniconda3/envs/nlp2023-hw2/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:110: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


[('<', (0, 1)), ('s', (1, 2)), ('>', (2, 3)), ('ĠA', (3, 5)), ('Ġsecond', (5, 12)), ('Ġtwitched', (12, 21)), ('Ġhis', (21, 25)), ('Ġshirtsleeve', (25, 37)), ('Ġ,', (37, 39)), ('Ġand', (39, 43)), ('Ġhe', (43, 46)), ('Ġfelt', (46, 51)), ('Ġa', (51, 53)), ('Ġbrief', (53, 59)), ('Ġburn', (59, 64)), ('Ġon', (64, 67)), ('Ġhis', (67, 71)), ('Ġupper', (71, 77)), ('Ġarm', (77, 81)), ('Ġ.', (81, 83)), ('ĠAnother', (83, 91)), ('Ġsnarled', (91, 99)), ('Ġclose', (99, 105)), ('Ġoverhead', (105, 114)), ('Ġ.', (114, 116)), ('Ġ</', (116, 119)), ('s', (119, 120)), ('>', (120, 121)), ('Ġsnarled', (121, 129)), (':', (129, 130)), ('Ġtwist', (130, 136)), ('Ġtogether', (136, 145)), ('Ġor', (145, 148)), ('Ġentwine', (148, 156)), ('Ġinto', (156, 161)), ('Ġa', (161, 163)), ('Ġconfusing', (163, 173)), ('Ġmass', (173, 178)), ('Ġ</', (178, 181)), ('s', (181, 182)), ('>', (182, 183))]
[('<', (0, 1)), ('s', (1, 2)), ('>', (2, 3)), ('ĠFirst', (3, 9)), ('Ġof', (9, 12)), ('Ġall', (12, 16)), ('Ġthere', (16, 22)), ('Ġwas

Using bos_token, but it is not set yet.
Using eos_token, but it is not set yet.
/home/francesco/miniconda3/envs/nlp2023-hw2/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:682: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  return trainer_fn(*args, **kwargs)


In [15]:
sentence = "The bank can guarantee deposits will eventually cover future tuition costs because it invests in adjustable-rate mortgage securities."
sense = "bank%1:14:00::"
definition = "a financial institution that accepts deposits and channels the money into lending activities"
max_seq_length = 128
target_idx = 1

tokenizer = AutoTokenizer.from_pretrained("roberta-base",  do_lower_case=True)

tokens_a = tokenizer.tokenize(sentence)
tokens_b = tokenizer.tokenize(definition)

tokens = [config.CLS_TOKEN] + tokens_a + ["[SEP]"]
segment_ids = [0] * len(tokens)
tokens += tokens_b + ["[SEP]"]
segment_ids += [1] * (len(tokens_b) + 1)

input_ids = tokenizer.convert_tokens_to_ids(tokens)

# The mask has 1 for real tokens and 0 for padding tokens. Only real
# tokens are attended to.
input_mask = [1] * len(input_ids)

padding = [0] * (max_seq_length - len(input_ids))
input_ids += padding
input_mask += padding
segment_ids += padding

from transformers import BertForTokenClassification
bert = BertForTokenClassification.from_pretrained("roberta-base", num_labels=2)


You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at roberta-base were not used when initializing BertForTokenClassification: ['roberta.encoder.layer.4.output.dense.bias', 'roberta.encoder.layer.8.output.LayerNorm.weight', 'roberta.encoder.layer.11.attention.self.value.weight', 'roberta.embeddings.LayerNorm.bias', 'roberta.encoder.layer.3.output.dense.weight', 'roberta.encoder.layer.4.intermediate.dense.weight', 'roberta.encoder.layer.7.attention.self.query.bias', 'roberta.encoder.layer.10.attention.output.dense.weight', 'roberta.encoder.layer.1.output.LayerNorm.bias', 'roberta.encoder.layer.1.intermediate.dense.bias', 'roberta.encoder.layer.4.intermediate.dense.bias', 'roberta.encoder.layer.4.output.dense.weight', 'roberta.encoder.layer.2.attention.self.query.bias', 'roberta.encoder.layer.10.output.LayerNorm.weight', 'roberta.encoder.layer.0.attent